In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import numpy as np
import pandas as pd
import scipy.stats

In [ ]:
# Demo:  gaussian mixture model for one-dimensional data;
# find maximum-likelihood parameters by writing a loss function
# and optimizing it.
# Fit constant model (SSE) that maximimzes SSE - likelihood of the data
# Fit normal model (mean, stddev) that maximize likelihood of the data
# Fit 50/50 gaussian mixture model to maximize likelihood of the data
# construct 2d contour maps of the likelihood function as a function of mu1, mu2, sigma1, sigma2
# Examine simple 1d Bayesian classifier for this 1d dataset.
# Construct an ROC curve for the above Bayesian classifier.

In [ ]:
# Here I load in a much-loved dataset from 1888, Francis Galton's heights of 900
# adults and their parents.

galton = pd.read_csv("../data/galton.csv")
galton.head()
y = galton.childHeight.values

In [ ]:
galton.head()

In [ ]:
galton.family.value_counts()

In [ ]:
plt.plot(galton.index, galton.childHeight)

In [ ]:
plt.plot(galton.index, galton.father)

In [ ]:
plt.plot(galton.index, galton.mother)

In [ ]:
plt.scatter(galton.father, galton.mother)

In [ ]:
plt.hist2d(galton.father, galton.mother, bins=[np.arange(62.5,77.5, 1), np.arange(57.5, 71, 1)])

In [ ]:
plt.hist(galton.childHeight, bins=np.arange(55, 80) + .5) 
plt.xlabel("Height in inches")
plt.ylabel("Number")

To make sure I can use optimization, let's try a test.

$$ LOSS_{SSE}(\theta; y)  = \sum (y-\theta)^2 $$

$$ \hat{\theta} = argmin_{\theta} \ \  LOSS_{SSE} (\theta; y) $$

If I wanted to replace all my y with a constant,
what number gives me the smallest summed error?



In [ ]:
# Here I define an extremely simple function.
# This function sums the squared-differences between
# each value in y and the paramter theta.
def LOSS_SSE( theta ):
    assert len(theta) == 1
    return np.sum( (y - theta)**2 ) 

In [ ]:
LOSS_SSE([64]) , LOSS_SSE([65]), LOSS_SSE([67]),  LOSS_SSE([68]),  LOSS_SSE([69])

In [ ]:
thetahat = minimize(LOSS_SSE, 0)
thetahat

Declare victory, the value 66.75 inches minimizes the sum-squared differences from all 946 data points.

In [ ]:
y

In [ ]:
# A slightly more elaborate model: SUM normal logpdf  ( data,  mean,  stddev )  
def LOSS_NORMAL(parameter):
    return np.sum( -scipy.stats.norm.logpdf( y, loc=parameter[0], scale=parameter[1])  )  

In [ ]:
LOSS_NORMAL([66.7,3.5]), LOSS_NORMAL([66.7,4.5]), LOSS_NORMAL([66.7,5.5]), LOSS_NORMAL([66.7,8.5]), 

In [ ]:
# Does it look like it has a minimum ? 
LOSS_NORMAL([86.7,3.5]), LOSS_NORMAL([66.7,3.5]), LOSS_NORMAL([46.7,3.5])

In [ ]:
# Run library optimization function
minimize(LOSS_NORMAL, [66, 3.5], method="BFGS")

Declare victory, now we have a mean and standard deviation that maximize the likelihood
of the data given a normal distribution with location, scale parameters 66.745 and 3.577.

Wait just one minute.

In [ ]:
y.mean(), y.std()

In [ ]:
# Hmm. Right.  I have used a machine gun to shoot a squirrel.

In [ ]:
# Plot the Normal distribution fit and the data histogram:
x = np.arange(55,80,.1)
yhat_g = scipy.stats.norm.pdf(x, loc=66.7459, scale=3.577)
plt.hist(y, bins=np.arange(55, 80))
plt.plot(x,yhat_g *900)

To get some insight into this likelihood function, 
$$ \textrm{norm.logpdf}(x, mu, sigma) = c -  \log \sigma + {(x - \mu)^2 \over 2 \sigma^2} $$
Let's evaluate it on a 2d grid in $\mu$ and $\sigma$ and make contour plots:


In [ ]:
# This is going to evaluate our LOSS_NORMAL function on a grid
xgrid = np.arange(55,80,2) 
ygrid = np.arange(0.5, 20.0, 0.5)
xax, yax = np.meshgrid(xgrid, ygrid, indexing="ij")
z = np.zeros(xax.shape)
print(xax.shape, yax.shape, z.shape)
for i in range(len(xgrid)):
    for j in range(len(ygrid)):
        z[i,j]= LOSS_NORMAL((xgrid[i], ygrid[j]))


In [ ]:
# Arright, I can't find how to plot the axes right without plt.contour and plt.contourf

In [ ]:
plt.contour(xax, yax, np.log(z), levels=30)
plt.xlabel("posterior mean")

plt.ylabel("posterior std")


In [ ]:
# It has an optimum.  (This is good.)

You didn't really need a loss function to find mean and standard deviation of a
collection of points.  

Now consider this model:

$$ P(x, \mu_1, \sigma_1, \mu_2, \sigma_2) = {1\over 2} \mathcal{N} (x; \mu_1, \sigma_1) + 
{1\over 2} \mathcal{N} (x; \mu_2, \sigma_2)  $$

This is a mixture of two Gaussian distributions.

In [ ]:
# So we construct the sum over all the data points of the log of two normal pdfs:

def LOSS_NORMAL2(parameter2):
    '''Sums likelihoods over y (assumed already defined) 
    for sum-of-two-equally-weighted-normal-distributions
    with paramters mu_1, sigma_1, mu_2, and sigma_2.'''
    assert len(parameter2) ==4  # throw an error if parameter2 has the wrong type
    return np.sum( -np.log(
                           scipy.stats.norm.pdf( y, loc=parameter2[0], scale=parameter2[1]) +
                           scipy.stats.norm.pdf( y, loc=parameter2[2], scale=parameter2[3]) ))  

In [ ]:
LOSS_NORMAL2([60,3.5,70,3.5])
    

In [ ]:
# Just to convince myself the sign is right.. 
LOSS_NORMAL2([0,3.5,70,3.5]), LOSS_NORMAL2([60,3.5,70,3.5]), LOSS_NORMAL2([120,3.5,70,3.5])


In [ ]:
minimize(LOSS_NORMAL2, [63, 2, 70, 2], method="BFGS")

In [ ]:
# And now evaluate this on a grid of len(xgrid2)x len(ygrid2) to make a contour plot
xgrid2 = np.arange(55,80,2) 
ygrid2 = np.arange(55,80,2)
xax2, yax2 = np.meshgrid(xgrid2, ygrid2, indexing="ij")
z2 = np.zeros(xax2.shape)
print(xax2.shape, yax2.shape, z.shape)
for i in range(len(xgrid2)):
    for j in range(len(ygrid2)):
        z2[i,j]= LOSS_NORMAL2((xgrid2[i],2.27,  ygrid2[j], 2.48))


In [ ]:
plt.contourf(xax2, yax2, np.log(z2), levels=30)
plt.xlabel("Mean 1 parameter")
plt.ylabel("Mean 2 parameter")
yax2

In [ ]:
# This is, believe it or not, reassuring.  There are two equal 
# optima, one with mean1 = 69 and mean2 = 64, and one with 
# mean1 = 64 and mean2 = 69; these correspond to switching the 
# labels between the large and the small groups.

Some questions for thought:  

* The optimizer claimed victory after only 105 evaluations of the loss function.  How many times did I evaluate the loss function to make these contour maps?

* The parameter space for my function was (mean1, std1, mean2, std2), that's four dimensions.
I plotted a two-dimensional slice, with std1 and std2 held fixed at their optmimum values.
Do you think std1 and std1 might be correlated with mean1 and mean2?

* There were two paramters (sigma_1 and sigma_2) that were .. "similar."   mean_1 and mean_2 were similar, but it was clear they would chase down different parts of the distribution.  But what does it mean if sigma_1 and sigma_2 are very different?   What would happen to the fitting process if I set sigma_2 = sigma_1 ? 

* There was one paramter that I failed to parameterize: the mixing coefficient.  In constructing the loss function I implicitly made the weights for normal 1 and normal 2 equal.    Would I get a better or worse fit if I let the algorithm fit the probability ratio between class 1 and class 2?



In [ ]:
# One more contour plotl this one looking at mean_1 and sigma_1 while
# the values of mean_2 and sigma_2 are kept fixed.
xgrid3 = np.arange(55,80,1) 
ygrid3 = np.arange(0.5,20,0.25)
xax3, yax3 = np.meshgrid(xgrid3, ygrid3, indexing="ij")
z3 = np.zeros(xax3.shape)
print(xax3.shape, yax3.shape, z.shape)
for i in range(len(xgrid3)):
    for j in range(len(ygrid3)):
        z3[i,j]= LOSS_NORMAL2((xgrid3[i],ygrid3[j] , 69.42351661 , 2.48))

In [ ]:
plt.contour(xax3, yax3, np.log(z3), levels=30)
plt.xlabel("Mean 1 parameter")
plt.ylabel("Std 1 parameter")
plt.savefig("2d-mu-sigma.png", dpi=300, bbox_inches="tight")
yax2

In [ ]:
# In this part of the likelihood function, there is a bit of 
# correlation between mu1 and sigma1. 
# Correlations are typical...
# But these have the effect that optimization of one axis at a time
# can be difficult, because the best fit in one direction spoil
# the fit in other directions.

In [ ]:
# Warning:  this is a 2d-plane slice through a 4-dimensional likelihood 
# function, holding mu_2 and sigma_2 constant.  In general, the optimium
# for mu_1 and sigma_1 is going to depend on these, so if I wanted to
# see the "real" joint distribution of mu_1 and sigma_1 I would need
# to find the optimium at each point or marginalize (by numerical 
# integration) to replace mu_2 and sigma_2 with probaiblity-density-informed
# expecatation values.

$$ P_{marginal}(x) = \int dy {P_{posterior}(x|y) P_{prior}(y) }$$

In [ ]:
# The two-paramter fit results were
thetahat = [64.08472207,  2.2771205 , 69.42351661,  2.48216295]
# and that was for the function 
yhat = 0.5 * scipy.stats.norm.pdf(x, loc=thetahat[0], scale=thetahat[1]) + 0.5 * scipy.stats.norm.pdf(x, loc=thetahat[2], scale=thetahat[3])

In [ ]:
x = np.arange(55,80,.1)
yhat_g = scipy.stats.norm.pdf(x, loc=66.7459, scale=3.577)
plt.hist(y,  bins=np.arange(55, 80))
plt.plot(x,yhat_g *900)
plt.plot(x, yhat*900)

In [ ]:
# So why does the data look like this?  


In [ ]:
galton.head(1)

In [ ]:
# Create a column whose value is 1 if gender is male, 0 otherwise: 
galton["indicator"] =  galton.gender == "male"

In [ ]:
galton.groupby(by="indicator").childHeight.hist(alpha=0.5, bins=np.arange(55, 80))
plt.xlabel("Child height (in)")

In [ ]:
# We can get the per-category mean and standard devation

# This goes by the jargon "Class-conditional distribution"
# which is to say, "All the data in class 1"

galton.groupby(by="indicator").childHeight.describe()

In [ ]:
# And can write down theoretical densities for normal distributions
# with the empirical mean and standard devation for the two classes:
density1 = scipy.stats.norm.pdf(x, loc=64.103974, scale=2.355653)
density2 = scipy.stats.norm.pdf(x, loc=69.234096, scale=2.623905)

In [ ]:
plt.figure(figsize=(9,4))
plt.subplot(121)
plt.hist(y,  bins=np.arange(55, 80))
plt.plot(x, np.array([(density1+density2)*450 ]).T ) 
plt.subplot(122)
plt.plot(x, np.array([density1*450, density2*450 ]).T ) 
galton.groupby(by="indicator").childHeight.hist(alpha=0.5, bins=np.arange(55, 80))


In [ ]:
# The best decision I can make is to pick the most likely class for each 
# value of x
x[np.min(np.where(density1<density2))]

In [ ]:
# And if I classify everyone shorter than 67.7 as female, 
# this is how my predictions compare to the original labels:
galton["classification"] = galton.childHeight > 67.7 
galton.groupby(["classification", "indicator"]).indicator.count()

In [ ]:

#  Correct: 773    Incorrect:  161 
#  Errors incorrectly classified as M : 32
#         incorrectly classified as F: 129 

len(galton), galton.indicator.sum(), galton.indicator.sum()/len(galton)

In [ ]:
# overall accuracy, adjusted for nothing:
773/934

Bayes' rule for inferring probability of class $\mathcal{C}_k $  ($k$ is the index that counts $r$ classes) given data $x$: 

$$ P(\mathcal{C}_k | x) \propto {P(x| \mathcal{C}_k) P(\mathcal{C}_k)}  $$

And the probability that $x$ came from class  $\mathcal{C}_1 $ is

$$ P(\mathcal{C}_1 | x) = {P(x| \mathcal{C}_1) P(\mathcal{C}_1) \over 
\displaystyle\sum_{i=1}^r P(x| \mathcal{C}_i) P(\mathcal{C}_i) }  $$


In [ ]:
# If I want a graph of my model's posterior probability of being
# in class 1 or class 2 as a function of x: 

p_class1 = density1 /(density1 + density2)
p_class2 = density2 /(density1 + density2)

plt.plot(x,p_class1)
plt.plot(x,p_class2) 
plt.ylabel("Posterior probablity of class | x")
plt.xlabel("child Height (in)")

In [ ]:
plt.plot(x, np.array([density1*450, density2*450 ]).T ) 
galton.groupby(by="indicator").childHeight.hist(alpha=0.5, bins=np.arange(55, 80))
plt.plot([66.7,66.7], [0,80])
plt.xlabel("Child height (in)")

In [ ]:
def find_fpr_fnr(threshold):
    a = galton.query("(indicator == 0) & childHeight <= @threshold").childHeight.count()
    b = galton.query("(indicator == 0) & childHeight > @threshold").childHeight.count()
    c = galton.query("(indicator == 1) & childHeight <= @threshold").childHeight.count()
    d = galton.query("(indicator == 1) & childHeight > @threshold").childHeight.count()
    print(threshold, a,b,c,d)
    return (b / (a+b) , c / (c+d) ) 

In [ ]:
find_fpr_fnr(62)

In [ ]:
trange = range(50,80)
fprfnr = np.array([find_fpr_fnr(threshold) for threshold in trange])

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
plt.plot(fprfnr[:,0], 1-fprfnr[:,1]) 
for x,y,z in zip (fprfnr[:,0], 1-np.array(fprfnr[:,1]) , list(map(str, trange))):
    print (x,y,z)
    ax.text(x, y, z)
plt.xlim([0,1])
plt.ylim([0,1])

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
plt.step(fprfnr[:,0], 1-fprfnr[:,1]) 
for x,y,z in zip (fprfnr[:,0], 1-np.array(fprfnr[:,1]) , list(map(str, trange))):
    print (x,y,z)
    ax.text(x, y, z)
plt.xlim([0,1])
plt.ylim([0,1])
plt.xlabel("FPR", fontsize=14)
plt.ylabel("TPR", fontsize=14)
plt.title("ROC curve for Galton height data / gender", fontsize=18)

In [ ]:
find_fpr_fnr(66.7)